<a href="https://www.kaggle.com/code/lokerj/sn-pj-prediction-price?scriptVersionId=152805358" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
import numpy as np # linear algebra 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
df1 = pd.read_csv("../input/dt-location/DDproperty_2006_data_Extracted - Copy.csv")
df1 = df1.drop(['navlink'], axis = 1)
df1

In [ ]:
!pip install firebase-admin

In [ ]:
import firebase_admin
from firebase_admin import credentials

# ใส่ path ของไฟล์ service account key ของ Firebase
cred = credentials.Certificate("/kaggle/input/firebase/snpj1-35e30-firebase-adminsdk-bxufi-7f98a4d541.json")
firebase_admin.initialize_app(cred)


In [ ]:
df1 = df1.dropna(how='any',axis=0)
df1


https://www.kaggle.com/code/alexisbcook/nested-and-repeated-data

In [ ]:
df1['Location'] = df1['Location'].str.split(",").str[0]
# df['Location'] = df['Location'].str.split(",").str[0]
df1['Location'].str.split(',', 1, expand=True)
df1

In [ ]:
df1 = df1.dropna(how='any',axis=0)
df1

In [ ]:
x = df1[['Location','bed','bath','listingfloorarea','listingfloorarea2','Type']]
y = df1['Price']

In [ ]:
x['listingfloorarea'] = x['listingfloorarea'].str.replace(' .','')
x['listingfloorarea'] = x['listingfloorarea'].str.replace('3ร.วาที่ดิน','')
x['listingfloorarea'] = x['listingfloorarea'].str.replace(')','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace('฿','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace('/','')
x['listingfloorarea2'] = x['listingfloorarea2'].str.replace(',','')

In [ ]:
x['listingfloorarea'] = x['listingfloorarea'].astype(str).astype(int)
x['listingfloorarea2'] = x['listingfloorarea2'].astype(str).astype(int)

In [ ]:
x

In [ ]:
from sklearn.preprocessing import LabelEncoder

for c in x.columns:
    if x[c].dtype=='object': 
        lbl = LabelEncoder()
        lbl.fit(list(x[c].values))
        x[c] = lbl.transform(x[c].values)

In [ ]:

from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state = 42)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor

In [ ]:
lr = LinearRegression()

knn = KNeighborsRegressor(n_neighbors=10)

dt = DecisionTreeRegressor(max_depth = 3)

rf = RandomForestRegressor(max_depth = 3, n_estimators=500)

ada = AdaBoostRegressor( n_estimators=50, learning_rate =.1)

gbr = GradientBoostingRegressor(max_depth=2, n_estimators=100, learning_rate =.2)

xgb = XGBRegressor(max_depth = 5, n_estimators=50, learning_rate =.1)

cb = CatBoostRegressor(learning_rate =.01, max_depth =5, verbose = 0)

regressors = [('Linear Regression', lr), ('K Nearest Neighbours', knn),
               ('Decision Tree', dt), ('Random Forest', rf), ('AdaBoost', ada),
              ('Gradient Boosting Regressor', gbr), ('XGBoost', xgb), ('catboost', cb)]

In [ ]:

from sklearn.metrics import r2_score

for regressor_name, regressor in regressors:
 
    # Fit regressor to the training set
    regressor.fit(x_train, y_train)    
   
    # Predict 
    y_pred = regressor.predict(x_test)
    accuracy = round(r2_score(y_test,y_pred),2)*100
    
    

   
    # Evaluate  accuracy on the test set
    print('{:s} : {:.0f} %'.format(regressor_name, accuracy))
    
    

In [3]:
from google.cloud import storage

# กำหนด path ที่ต้องการบันทึกโมเดล
local_model_path = "Users\mangt\Desktop\SNPJ1"

# กำหนดชื่อ bucket ใน GCP
bucket_name = 'SNPJ1'

# กำหนดชื่อไฟล์โมเดลใน bucket
blob_name = 'snpj1-35e30'

# อัปโหลดไฟล์โมเดลไปยัง GCP bucket
client = storage.Client()
bucket = client.get_bucket(bucket_name)
blob = bucket.blob(blob_name)
blob.upload_from_filename(local_model_path)


DefaultCredentialsError: Could not automatically determine credentials. Please set GOOGLE_APPLICATION_CREDENTIALS or explicitly create credentials and re-run the application. For more information, please see https://cloud.google.com/docs/authentication/getting-started

In [ ]:
# import firebase_admin
# from firebase_admin import credentials, db

# # แทนที่ "path/to/your/serviceAccountKey.json" ด้วยที่อยู่และชื่อไฟล์จริงของไฟล์ service account key ที่คุณอัปโหลด
# cred = credentials.Certificate("/kaggle/input/firebase/snpj1-35e30-firebase-adminsdk-bxufi-7f98a4d541.json")
# firebase_admin.initialize_app(cred, {'databaseURL': 'https://snpj1-35e30-default-rtdb.asia-southeast1.firebasedatabase.app/'})

# # แทนที่ '/path/in/database' ด้วยพาธที่คุณต้องการเก็บข้อมูลใน Firebase Realtime Database
# path_in_database = 'https://snpj1-35e30-default-rtdb.asia-southeast1.firebasedatabase.app/'

# # แทนที่ 'key1', 'value1', 'key2', 'value2' ด้วยข้อมูลที่คุณต้องการส่ง
# data_to_send = {
#     'key1': 'value1',
#     'key2': 'value2',
#     # ...
# }

# # ส่งข้อมูลไปยัง Firebase Realtime Database
# ref = db.reference(path_in_database)
# ref.set(data_to_send)

In [ ]:
# # @save
# import tensorflow as tf

# model = tf.keras.models.Sequential([...])  # สร้างโมเดลของคุณที่นี่

# # Export โมเดลเป็น SavedModel
# tf.saved_model.save(model, '/kaggle/working/saved_model')